In [242]:
import pandas as pd
import numpy as np
from pandas import Timestamp
from pandas.tseries.offsets import DateOffset

In [243]:
weather_data = pd.read_csv('Maximum _ Minimum Temperature Data/WA/output_final.csv')

In [244]:
selected_station=pd.read_csv('Combined_temperature_data/WA/selected_station.csv')
selected_station['Selected starting date'] = pd.to_datetime(selected_station['Selected starting date'])
selected_station['Selected ending date'] = pd.to_datetime(selected_station['Selected ending date'])

In [245]:
weather_data=weather_data.drop(columns=['Unnamed: 0','Unnamed: 0.1'])

In [246]:
weather_data=weather_data.rename(columns={'Bureau of Meteorology station number':'Station Number'})

We only choose the data from selected stations list file

In [247]:
weather_data=weather_data[weather_data['Station Number'].isin(selected_station['Station Number'])]

In [248]:
weather_data

,Product code_x,Station Number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality_x,Product code_y,Minimum temperature (Degree C),Days of accumulation of minimum temperature,Quality_y,stationname,localgovernmentarea,latitude,longtitude
0,IDCJAC0010,10007,1957,1,1,28.3,NaN,Y,IDCJAC0011,15.6,NaN,Y,BENCUBBIN,Merredin,-30.8081,117.8603
1,IDCJAC0010,10007,1957,1,2,28.9,1.0,Y,IDCJAC0011,14.4,1.0,Y,BENCUBBIN,Merredin,-30.8081,117.8603
2,IDCJAC0010,10007,1957,1,3,32.2,1.0,Y,IDCJAC0011,14.4,1.0,Y,BENCUBBIN,Merredin,-30.8081,117.8603
3,IDCJAC0010,10007,1957,1,4,35.0,1.0,Y,IDCJAC0011,17.2,1.0,Y,BENCUBBIN,Merredin,-30.8081,117.8603
4,IDCJAC0010,10007,1957,1,5,32.2,1.0,Y,IDCJAC0011,18.3,1.0,Y,BENCUBBIN,Merredin,-30.8081,117.8603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093860,IDCJAC0010,9998,2020,8,2,15.2,1.0,N,IDCJAC0011,8.8,1.0,N,NORTH WALPOLE,Denmark,-34.9469,116.7222
2093861,IDCJAC0010,9998,2020,8,3,12.0,1.0,N,IDCJAC0011,6.8,1.0,N,NORTH WALPOLE,Denmark,-34.9469,116.7222
2093862,IDCJAC0010,9998,2020,8,4,13.5,1.0,N,IDCJAC0011,7.4,1.0,N,NORTH WALPOLE,Denmark,-34.9469,116.7222
2093863,IDCJAC0010,9998,2020,8,5,14.7,1.0,N,IDCJAC0011,11.4,1.0,N,NORTH WALPOLE,Denmark,-34.9469,116.7222


Let see how many NaN Maximum temperature row

In [249]:
len(weather_data[np.isnan(weather_data['Minimum temperature (Degree C)'])==True])

201714

We will use the data from the year of 2000

In [250]:
weather_data=weather_data[weather_data['Year']>=2000]

In [251]:
len(weather_data[np.isnan(weather_data['Maximum temperature (Degree C)'])==True])

47260

In [252]:
weather_data['Date Time']=pd.to_datetime(weather_data['Year'].astype(str)+'-'+weather_data['Month'].astype(str)+'-'+weather_data['Day'].astype(str))

c:\users\ngoct\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [253]:
mean_max_temp=weather_data.groupby(['Station Number','Month'])['Maximum temperature (Degree C)'].mean().reset_index()
mean_min_temp=weather_data.groupby(['Station Number','Month'])['Minimum temperature (Degree C)'].mean().reset_index()

In [254]:
for index,row in selected_station.iterrows():
    station_no=row['Station Number']
    end_date = row['Selected ending date']
    weather_data=weather_data.drop(weather_data[(weather_data['Station Number']==station_no) & (pd.to_datetime(weather_data['Date Time'])>pd.to_datetime(end_date))].index)
    

In [255]:
len(weather_data)

916915

In [256]:
temp_data = pd.DataFrame()
for station_number in selected_station['Station Number']:
    station_temp = weather_data[weather_data['Station Number']==station_number]
    if selected_station[selected_station['Station Number']==station_number]['Selected starting date'].values[0] < pd.to_datetime(selected_station[selected_station['Station Number']==station_number]['Starting Date recording min'].values[0]):
        idx = pd.date_range(pd.to_datetime( selected_station[selected_station['Station Number']==station_number]['Selected starting date'].values[0]),pd.to_datetime( selected_station[selected_station['Station Number']==station_number]['Selected ending date'].values[0]))
        station_temp=station_temp.set_index('Date Time').reindex(idx).reset_index().rename(columns={'index':'Date'})
        station_temp['Station Number']=station_temp['Station Number'].fillna(station_number)
        station_temp['Product code_x']=station_temp['Product code_x'].fillna(weather_data[weather_data['Station Number']==station_number]['Product code_x'].values[0])
        station_temp['stationname']=station_temp['stationname'].fillna(weather_data[weather_data['Station Number']==station_number]['stationname'].values[0])
        station_temp['latitude']=station_temp['latitude'].fillna(weather_data[weather_data['Station Number']==station_number]['latitude'].values[0])
        station_temp['longtitude']=station_temp['longtitude'].fillna(weather_data[weather_data['Station Number']==station_number]['longtitude'].values[0])
        station_temp['localgovernmentarea']=station_temp['localgovernmentarea'].fillna(weather_data[weather_data['Station Number']==station_number]['localgovernmentarea'].values[0])
    temp_data=pd.concat([temp_data,station_temp])

In [257]:
temp_data['Date Time']=temp_data['Date Time'].fillna(temp_data['Date'])
temp_data=temp_data.drop(columns=['Date'])

In [258]:
temp_data['Month'] = temp_data['Date Time'].dt.month

In [259]:
temp_data

,Product code_x,Station Number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality_x,Product code_y,Minimum temperature (Degree C),Days of accumulation of minimum temperature,Quality_y,stationname,localgovernmentarea,latitude,longtitude,Date Time
42806,IDCJAC0010,1006.0,2000.0,1,1.0,NaN,NaN,NaN,IDCJAC0011,NaN,NaN,NaN,WYNDHAM AERO,Wyndham-East Kimberley,-15.5100,128.1503,2000-01-01
42807,IDCJAC0010,1006.0,2000.0,1,2.0,NaN,NaN,NaN,IDCJAC0011,NaN,NaN,NaN,WYNDHAM AERO,Wyndham-East Kimberley,-15.5100,128.1503,2000-01-02
42808,IDCJAC0010,1006.0,2000.0,1,3.0,NaN,NaN,NaN,IDCJAC0011,NaN,NaN,NaN,WYNDHAM AERO,Wyndham-East Kimberley,-15.5100,128.1503,2000-01-03
42809,IDCJAC0010,1006.0,2000.0,1,4.0,NaN,NaN,NaN,IDCJAC0011,NaN,NaN,NaN,WYNDHAM AERO,Wyndham-East Kimberley,-15.5100,128.1503,2000-01-04
42810,IDCJAC0010,1006.0,2000.0,1,5.0,NaN,NaN,NaN,IDCJAC0011,NaN,NaN,NaN,WYNDHAM AERO,Wyndham-East Kimberley,-15.5100,128.1503,2000-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837035,IDCJAC0010,200790.0,2020.0,8,2.0,25.7,1.0,N,IDCJAC0011,22.2,1.0,N,CHRISTMAS ISLAND AERO,NaN,-10.4528,105.6875,2020-08-02
837036,IDCJAC0010,200790.0,2020.0,8,3.0,26.2,1.0,N,IDCJAC0011,22.3,1.0,N,CHRISTMAS ISLAND AERO,NaN,-10.4528,105.6875,2020-08-03
837037,IDCJAC0010,200790.0,2020.0,8,4.0,26.5,1.0,N,IDCJAC0011,22.9,1.0,N,CHRISTMAS ISLAND AERO,NaN,-10.4528,105.6875,2020-08-04
837038,IDCJAC0010,200790.0,2020.0,8,5.0,26.8,1.0,N,IDCJAC0011,22.4,1.0,N,CHRISTMAS ISLAND AERO,NaN,-10.4528,105.6875,2020-08-05


In [260]:
missing_max_data = temp_data[np.isnan(temp_data['Maximum temperature (Degree C)'])==True]

In [261]:
len(missing_max_data)

55271

In [262]:
missing_min_data= temp_data[np.isnan(temp_data['Minimum temperature (Degree C)'])==True]

In [263]:
len(missing_min_data)

60414

In [264]:
def filling_missing_temp(row,col_name,mean_temp):
    if np.isnan(row[col_name]) == True:
        station_no= row['Station Number']
        date = pd.to_datetime(row['Date Time'])
        month=row['Month']
        starting_date =  pd.to_datetime(selected_station[selected_station['Station Number'] == station_no]['Selected starting date'].values[0])
        ending_date = pd.to_datetime(selected_station[selected_station['Station Number'] == station_no]['Selected ending date'].values[0])
        if date.strftime('%Y') ==  starting_date.strftime('%Y'):
            date_offset=date + DateOffset(years=1)
            while date_offset <= ending_date:
                nextyear_temp = temp_data[(temp_data['Station Number']==station_no) & (temp_data["Date Time"]==date_offset)][col_name].values[0]
                if np.isnan(nextyear_temp) == False:
                    return nextyear_temp
                date_offset=date_offset + DateOffset(years=1)
                
            return mean_temp.loc[(mean_temp['Station Number']==station_no) & (mean_temp['Month']==month)][col_name].values[0]
        elif date.strftime('%Y') ==  ending_date.strftime('%Y'):
            date_offset=date - DateOffset(years=1)
            while date_offset >= starting_date:
                prevyear_temp = temp_data[(temp_data['Station Number']==station_no) & (temp_data["Date Time"]==date_offset)][col_name].values[0]
                if np.isnan(prevyear_temp) == False:
                    return prevyear_temp
                date_offset=date_offset - DateOffset(years=1)
                
            return mean_temp.loc[(mean_temp['Station Number']==station_no) & (mean_temp['Month']==month)][col_name].values[0]
        else:
            date_offset=date + DateOffset(years=1)
            nextyear_null=True
            nextyear_temp=-1
            
            while date_offset <= ending_date:
                nextyear_temp = temp_data[(temp_data['Station Number']==station_no) & (temp_data["Date Time"]==date_offset)][col_name].values[0]
                if np.isnan(nextyear_temp) == False:
                    nextyear_null=False
                    break
                date_offset=date_offset + DateOffset(years=1)
            prevyear_temp=-1
            prevyear_null=True
            date_offset=date - DateOffset(years=1)
            while date_offset >= starting_date:
                prevyear_temp = temp_data[(temp_data['Station Number']==station_no) & (temp_data["Date Time"]==date_offset)][col_name].values[0]
                if np.isnan(prevyear_temp) == False:
                    prevyear_null=False
                    break
                date_offset=date_offset - DateOffset(years=1)
                
            if prevyear_null== True and nextyear_null== True:
                return mean_temp.loc[(mean_temp['Station Number']==station_no) & (mean_temp['Month']==month)][col_name].values[0]
            elif prevyear_null ==True and nextyear_null == False:
                return nextyear_temp
            elif prevyear_null ==False and nextyear_null == True:
                return prevyear_temp
            else:
                retun_tem=(nextyear_temp+prevyear_temp)/2
                return retun_tem

    else:
        return row[col_name]
        
    

In [265]:
temp_data['Maximum temperature (Degree C)']=temp_data.apply(lambda row : filling_missing_temp(row,'Maximum temperature (Degree C)',mean_max_temp),axis=1)

In [266]:
temp_data['Minimum temperature (Degree C)']=temp_data.apply(lambda row : filling_missing_temp(row,'Minimum temperature (Degree C)',mean_min_temp),axis=1)

In [237]:
missing_min_data = temp_data[np.isnan(temp_data['Maximum temperature (Degree C)'])==True]
len(missing_min_data)

0

In [238]:
missing_min_data = temp_data[np.isnan(temp_data['Minimum temperature (Degree C)'])==True]
len(missing_min_data)

0

In [240]:
temp_data

,Product code_x,Station Number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality_x,Product code_y,Minimum temperature (Degree C),Days of accumulation of minimum temperature,Quality_y,stationname,localgovernmentarea,latitude,longtitude,Date Time
42806,IDCJAC0010,1006.0,2000.0,1,1.0,38.0,NaN,NaN,IDCJAC0011,26.0,NaN,NaN,WYNDHAM AERO,Wyndham-East Kimberley,-15.5100,128.1503,2000-01-01
42807,IDCJAC0010,1006.0,2000.0,1,2.0,39.0,NaN,NaN,IDCJAC0011,27.0,NaN,NaN,WYNDHAM AERO,Wyndham-East Kimberley,-15.5100,128.1503,2000-01-02
42808,IDCJAC0010,1006.0,2000.0,1,3.0,40.0,NaN,NaN,IDCJAC0011,27.0,NaN,NaN,WYNDHAM AERO,Wyndham-East Kimberley,-15.5100,128.1503,2000-01-03
42809,IDCJAC0010,1006.0,2000.0,1,4.0,40.0,NaN,NaN,IDCJAC0011,27.0,NaN,NaN,WYNDHAM AERO,Wyndham-East Kimberley,-15.5100,128.1503,2000-01-04
42810,IDCJAC0010,1006.0,2000.0,1,5.0,39.0,NaN,NaN,IDCJAC0011,28.0,NaN,NaN,WYNDHAM AERO,Wyndham-East Kimberley,-15.5100,128.1503,2000-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837035,IDCJAC0010,200790.0,2020.0,8,2.0,25.7,1.0,N,IDCJAC0011,22.2,1.0,N,CHRISTMAS ISLAND AERO,NaN,-10.4528,105.6875,2020-08-02
837036,IDCJAC0010,200790.0,2020.0,8,3.0,26.2,1.0,N,IDCJAC0011,22.3,1.0,N,CHRISTMAS ISLAND AERO,NaN,-10.4528,105.6875,2020-08-03
837037,IDCJAC0010,200790.0,2020.0,8,4.0,26.5,1.0,N,IDCJAC0011,22.9,1.0,N,CHRISTMAS ISLAND AERO,NaN,-10.4528,105.6875,2020-08-04
837038,IDCJAC0010,200790.0,2020.0,8,5.0,26.8,1.0,N,IDCJAC0011,22.4,1.0,N,CHRISTMAS ISLAND AERO,NaN,-10.4528,105.6875,2020-08-05


In [241]:
temp_data.to_csv('Maximum _ Minimum Temperature Data/WA/wa_temperature_data.csv',index=False)